In [1]:
from DataGenerator import DataGenerator

data = DataGenerator()

In [ ]:
data.test

In [ ]:
data.train

In [2]:
import tensorflow as tf
X = data.add_negatives(data.train, n_samples=4)

X_train = data.get_features(X)
# X_train = tf.convert_to_tensor(data.get_features(X))
y_train = data.get_target(X)
# y_train = tf.convert_to_tensor(data.get_target(X))

In [3]:
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, multiply
from tensorflow.keras.regularizers import l2

def GMF(n_users, n_items, latent_dim):
    user_input = Input(shape=(1,), dtype='int32', name='user')
    item_input = Input(shape=(1,), dtype='int32', name='movie')

    user_embedding = Embedding(input_dim = n_users,
                                    output_dim = latent_dim,
                                    name = 'user_embedding',
                                    embeddings_regularizer = l2(0),
                                    input_length=1)
    item_embedding = Embedding(input_dim = n_items,
                                output_dim = latent_dim,
                                name = 'item_embedding',
                                # init = init_normal,
                                embeddings_regularizer = l2(0),
                                input_length=1)
    user_latent = Flatten()(user_embedding(user_input))
    item_latent = Flatten()(item_embedding(item_input))
    predict = multiply([user_latent, item_latent])
    prediction = Dense(1, activation=tf.nn.sigmoid, kernel_initializer='lecun_uniform')(predict)
    return tf.keras.Model(inputs=[user_input, item_input],
                  outputs=prediction)

In [4]:
learning_rate=0.001
gmf = GMF(data.num_users, data.num_movies, latent_dim=8)
gmf.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=learning_rate),
            loss='binary_crossentropy')

2021-10-23 10:20:35.851404: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def find_batch_size():
    if len(tf.config.list_physical_devices('GPU'))>0:
        return 1024
    else:
        return 64
batch_size = find_batch_size()

In [8]:
# for e in range(5):
#     gmf.fit(X_train, y_train, batch_size=batch_size, epochs=1, verbose=1, shuffle=True)



X_test

NameError: name 'X_test' is not defined

In [10]:
gmf.fit(X_train, y_train.T)

    1/31823 [..............................] - ETA: 45:36 - loss: 0.6930

InvalidArgumentError:  indices[7,0] = 3752 is not in [0, 3706)
	 [[node model/item_embedding/embedding_lookup (defined at var/folders/kf/j1jq_1qs78n4gd1vczc8vfw80000gn/T/ipykernel_10631/1528553782.py:1) ]] [Op:__inference_train_function_742]

Errors may have originated from an input operation.
Input Source operations connected to node model/item_embedding/embedding_lookup:
 model/item_embedding/embedding_lookup/480 (defined at Users/franciscocancedda/.pyenv/versions/3.9.5/lib/python3.9/contextlib.py:117)

Function call stack:
train_function


In [ ]:
D = data.get_dataset(X)
D = D.shuffle(buffer_size=100)
D = D.batch(batch_size)

In [ ]:
gmf.fit(D)

In [ ]:
for epoch in range(2):
    for batch in D:
        print(batch.shape)
    print("End of epoch: ", epoch)

In [ ]:
next(iter(D))